In [1]:
import csv
import json
import re
import ast

import pandas as pd
import requests

In [2]:
def getItem(category):
    url = 'https://api.nookplaza.net/items?category={}'.format(category)
    r = requests.get(url)
    text = r.json()
    return text['results']

In [4]:
nookplazaUrl = 'https://nookplaza.net'
r = requests.get(nookplazaUrl)

m = re.search('main\.dart\.js\?v=(\d*)', r.text)
dartJs = m.group()

mainJsUrl = '{}/{}'.format(nookplazaUrl, dartJs)
r = requests.get(mainJsUrl)
mainJs = r.text

# 'C.Iz=new H.b(13,{},C.a,[P.a,P.a])'
m = re.findall('new H\.b\(13,(.*),C\.a,\[P\.a,P\.a\]\)', mainJs)

m = [item.encode("utf-8").decode('unicode-escape') for item in m]
m = [ast.literal_eval(item) for item in m if not 'null' in item]

nameMap = {}
for item in m:
    nameMap[item["en-us"]] = item

print('get {} i18n data'.format(len(nameMap.keys())))

df = pd.read_excel('../tempData/Furniture.xlsx', sheetname=3)

EnTwTable = ['類別', '取得方式', '標籤', '主題', '款式', '樣式', '款式說明', '樣式說明', 'DIY相關', '藝術品', '作者/年代/材質', '藝術品真實圖']

nameMapCustom = {}
for EnTw in EnTwTable:
    itemType_en = df[EnTw].tolist()
    itemType_tw = df['{}_tw'.format(EnTw)].tolist()
    itemType_en = [x for x in itemType_en if str(x) != 'nan']
    itemType_tw = [x for x in itemType_tw if str(x) != 'nan']
    
    for item in zip(itemType_en, itemType_tw):
        if item[0] in nameMapCustom and item[1] != nameMapCustom[item[0]]['zh-tw']:
            print(item)
            print(nameMapCustom[item[0]])
            print('------\n')
    
        nameMapCustom[item[0]] = {
            'zh-tw': item[1]
        }
        
print('Custom {} i18n data'.format(len(nameMapCustom.keys())))

def en2twFn(en, Custom = False):
    if Custom:
        if en in nameMapCustom:
            return nameMapCustom[en]

        elif en.lower() in nameMapCustom:
            return nameMapCustom[en.lower()]

        elif (en[0].lower() + en[1:]) in nameMapCustom:
            return nameMapCustom[(en[0].lower() + en[1:])]
    
    if en.lower() in nameMap:
        return nameMap[en.lower()]

    elif (en[0].lower() + en[1:]) in nameMap:
        return nameMap[(en[0].lower() + en[1:])]
    
    elif en in nameMap:
        return nameMap[en]
    
    elif en in nameMapCustom:
            return nameMapCustom[en]

    elif en.lower() in nameMapCustom:
        return nameMapCustom[en.lower()]

    elif (en[0].lower() + en[1:]) in nameMapCustom:
        return nameMapCustom[(en[0].lower() + en[1:])]
    
    print(en)
    raise en
#         return {'zh-tw': en}

get 5713 i18n data
Custom 1036 i18n data


In [4]:
en2twFn('Wistful painting')

{'de-de': 'Perlengemälde',
 'en-gb': 'wistful painting',
 'en-us': 'wistful painting',
 'es-es': 'retrato perla',
 'fr-ca': 'toile nacrée',
 'fr-fr': 'toile nacrée',
 'it-it': 'quadro realista',
 'ja-jp': 'すてきなめいが',
 'ko-kr': '근사한 명화',
 'nl-nl': 'melancholische schilderij',
 'ru-ru': 'мечтательная картина',
 'zh-cn': '漂亮的名画',
 'zh-tw': '漂亮的名畫'}

In [6]:
output = []
furnitures = getItem('Furniture')
recipes = getItem('Recipes')

DIYMap = {}
ArtMap = {}
for diy in recipes:
    DIYMap[diy['name']] = diy['content']

for item in furnitures:
    if item["category"] == "Art":
        pickArt = {}
        name = en2twFn(item['name'].replace(' (fake)', ''))

        pickArt['id'] = int(item['content']['internalID'])
        pickArt['name_c'] = name['zh-tw']
        pickArt['name_j'] = name['ja-jp']
        pickArt['name_e'] = name['en-us']
        
        pickArt['artist'] = en2twFn(item['content']['artist'])['zh-tw']
        pickArt['filename'] = item['content']['filename']
        pickArt['interact'] = item['content']['interact']
        pickArt['itemCategory'] = item['content']['itemCategory']
        pickArt['museumDescription'] = item['content']['museumDescription']
        pickArt['obtainedFrom'] = '、'.join([en2twFn(item)['zh-tw'] for item in item['content']['obtainedFrom']])
        pickArt['realArtworkTitle'] = item['content']['realArtworkTitle']
        pickArt['realArtworkTitle_tw'] = en2twFn(item['content']['realArtworkTitle'])['zh-tw']
        pickArt['tag'] = en2twFn(item['content']['tag'], True)['zh-tw']
        pickArt['themes'] = [en2twFn(theme, True)['zh-tw'] for theme in item['content']['themes'] if theme != 'None']
        pickArt['size'] = '{}x{}'.format(item['content']['size']['rows'], item['content']['size']['cols'])
        pickArt['realArtImage'] = en2twFn(item['content']['filename'].replace('Fake', ''))['zh-tw']
        
        if not name['en-us'] in ArtMap:
            ArtMap[name['en-us']] = {}
            
        ArtMap[name['en-us']][str(item['content']['genuine'])] = pickArt
        continue
    
#     print(item['name'])
#     print('')
    pickData = {}
    name = en2twFn(item['name'])
    
    pickData['id'] = int(item['content']['internalID'])
    pickData['name_c'] = name['zh-tw']
    pickData['name_j'] = name['ja-jp']
    pickData['name_e'] = name['en-us']
    pickData['buy'] = item['content']['buy'] if 'buy' in item['content'] else None
    pickData['sell'] = item['content']['sell']

    pickData['category'] = en2twFn(item['content']['category'], True)['zh-tw']
    pickData['tag'] = en2twFn(item['content']['tag'], True)['zh-tw']
    pickData['themes'] = [en2twFn(theme, True)['zh-tw'] for theme in item['content']['themes'] if theme != 'None']
    
    pickData['interact'] = item['content']['interact']
    pickData['size'] = '{}x{}'.format(item['content']['size']['rows'], item['content']['size']['cols'])
    pickData['obtainedFrom'] = '、'.join([en2twFn(item)['zh-tw'] for item in item['content']['obtainedFrom']])
    pickData['DIY'] = item['content']['dIY']
#     pickData['diyInfo'] = {}
    pickData['diyInfoMaterials'] = []
    pickData['diyInfoObtainedFrom'] = []
    pickData['diyInfoSourceNotes'] = ''

    if item['content']['dIY']:
        diyInfo = DIYMap[item['name']]
        pickData['diyInfoMaterials'] = [{'count': item['count'], 'itemName': en2twFn(item['itemName'])['zh-tw']} for item in diyInfo['materials']]
        pickData['diyInfoObtainedFrom'] = [en2twFn(obtainedFrom, True)['zh-tw'] for obtainedFrom in diyInfo['obtainedFrom']]
        pickData['diyInfoSourceNotes'] = '' if diyInfo['sourceNotes'] == '' else en2twFn(diyInfo['sourceNotes'], True)['zh-tw']
        
    pickData['filename'] = item['content']['filename']
    
    if 'bodyTitle' in item['content'] and item['content']['bodyTitle']:
        pickData['bodyTitle'] = en2twFn(item['content']['bodyTitle'], True)['zh-tw']
    else:
        pickData['bodyTitle'] = '無'
        
    if 'bodyCustomize' in item['content']:
        pickData['bodyCustomize'] = item['content']['bodyCustomize']
    else:
        pickData['bodyCustomize'] = False
    
    if 'patternTitle' in item['content'] and item['content']['patternTitle']:
        pickData['patternTitle'] = en2twFn(item['content']['patternTitle'], True)['zh-tw']
    else:
        pickData['patternTitle'] = '無'
        
    if 'patternCustomize' in item['content']:
        pickData['patternCustomize'] = item['content']['patternCustomize']
    else:
        pickData['patternCustomize'] = False

    variations = {
        "bodys": {},
        "pattrens": {}
    }
    for variation in item['variations']:
        content = variation['content']
        bodyIndex, pattrenIndex = content['filename'].split('_')[-2:]
        
        bodyColor = en2twFn(content['bodyColor'], True)['zh-tw'] if 'bodyColor' in content and content['bodyColor'] is not None else ''
        patternColor = en2twFn(content['patternColor'], True)['zh-tw'] if 'patternColor' in content and content['patternColor'] is not None else ''
        
        variations['bodys'][bodyColor] = bodyIndex
        variations['pattrens'][patternColor] = pattrenIndex
        
    pickData['variations'] = variations
    output.append(pickData)

output = sorted(output, key=lambda k: k['id'])

In [7]:
# print(ArtMap)

artOutput = []
for name in ArtMap.keys():
#     print(name)
#     print(ArtMap[name]['True'])
    if 'False' in ArtMap[name]:
        ArtMap[name]['True']['HaveFake'] = True
    else:
        ArtMap[name]['True']['HaveFake'] = False
    artOutput.append(ArtMap[name]['True'])

artOutput = sorted(artOutput, key=lambda k: k['id'])
with open('../../nook_link/src/data/art.json', 'wt') as fout:
    fout.write(json.dumps(artOutput))
    
with open('art.json', 'wt') as fout:
    fout.write(json.dumps(artOutput))
    
dictName = ['id', 'name_e', 'name_j', 'name_c', 'realArtworkTitle', 'realArtworkTitle_tw', 'artist', 'museumDescription']
dictName += ['obtainedFrom', 'filename', 'HaveFake', 'interact', 'itemCategory', 'themes', 'tag', 'size', 'realArtImage']

with open('art.csv', 'wt', encoding='utf-8', newline='') as fout:
    csvout = csv.writer(fout)
    
    output_csv = [dictName]
    for row in artOutput:
        row_ = [row[name] for name in dictName]
        output_csv.append(row_)
    csvout.writerows(output_csv)

In [56]:
with open('../../nook_link/src/data/furniture.json', 'wt') as fout:
    fout.write(json.dumps(output))

with open('furniture.json', 'wt') as fout:
    fout.write(json.dumps(output))
    
dictName = ['id', 'name_e', 'name_j', 'name_c', 'buy', 'sell', 'category', 'tag', 'themes', 'interact', 'size', 'obtainedFrom', 'DIY', 'diyInfoMaterials', 'diyInfoObtainedFrom', 'diyInfoSourceNotes']
dictName += ['filename', 'bodyTitle', 'patternTitle', 'bodyCustomize', 'patternCustomize', 'variations']

with open('items.csv', 'wt', encoding='utf-8', newline='') as fout:
    csvout = csv.writer(fout)
    
    output_csv = [dictName]
    for row in output:
        row_ = [row[name] for name in dictName]
        output_csv.append(row_)
    csvout.writerows(output_csv)

In [59]:
output = []

recipes = getItem('Recipes')

for item in recipes:

#     print(item['name'])
#     print('')
    pickData = {}
    name = en2twFn(item['name'])

    pickData['id'] = int(item['content']['internalID'])
    pickData['name_c'] = name['zh-tw']
    pickData['name_j'] = name['ja-jp']
    pickData['name_e'] = name['en-us']
    
    pickData['category'] = en2twFn(item['content']['itemCategory'], True)['zh-tw']
    pickData['filename'] = item['content']['itemImage'].split('/')[-1].replace('.png', '')
    pickData['DIY'] = True
    pickData['sell'] = item['content']['sell']
    pickData['materials'] = [{'count': item['count'], 'itemName': en2twFn(item['itemName'])['zh-tw']} for item in item['content']['materials']]
    pickData['obtainedFrom'] = [en2twFn(obtainedFrom, True)['zh-tw'] for obtainedFrom in item['content']['obtainedFrom']]
    pickData['sourceNotes'] = '' if item['content']['sourceNotes'] == '' else en2twFn(item['content']['sourceNotes'], True)['zh-tw']
    output.append(pickData)

output = sorted(output, key=lambda k: k['id'])

with open('../../nook_link/src/data/recipes.json', 'wt') as fout:
    fout.write(json.dumps(output))
    
with open('recipes.json', 'wt') as fout:
    fout.write(json.dumps(output))
    
dictName = ['id', 'name_e', 'name_j', 'name_c', 'category', 'filename', 'sell', 'materials', 'obtainedFrom', 'sourceNotes']

with open('recipes.csv', 'wt', encoding='utf-8', newline='') as fout:
    csvout = csv.writer(fout)
    
    output_csv = [dictName]
    for row in output:
        row_ = [row[name] for name in dictName]
        output_csv.append(row_)
    csvout.writerows(output_csv)